In [ ]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
def SGD_regression(epochs, B, time_steps, train_images, train_labels, step_size = 0.01):
  """
  Stochastic Gradient Descent (SGD) for logistic regression.

  Parameters:
  - epochs (int): Number of training epochs.
  - B (int): Batch size for each iteration.
  - time_steps (int): Number of time steps to record for timing.
  - train_images (numpy.ndarray): Input features for training.
  - train_labels (numpy.ndarray): Output labels for training.
  - step_size (float, optional): Learning rate for SGD. Default is 0.01.

  Returns:
  - weights (numpy.ndarray): Learned weights after training.
  - loss (numpy.ndarray): Array of losses at each epoch.
  - times (list): List of time passed for each time step.
  """
  #calculating output by logistic regression
  def calculate_output(x, w):
    exponent = np.clip(-x @ w, -700, 700)  # Clip to prevent overflow/underflow
    return 1 / (1 + np.exp(exponent))

  #calculate loss
  def calculate_loss(x, w, y):
    y_hat = calculate_output(x, w)

    epsilon = 1e-15
    y_hat = np.clip(y_hat, epsilon, 1 - epsilon)
    return -1/len(y_hat) * np.sum(y.T @ np.log(y_hat) + (1-y).T @ np.log(1 - y_hat))

  #initialize start time for timing
  start = time.time()
  times = [0]

  weights = np.zeros((train_images.shape[1], 1))
  N = train_images.shape[0]

  loss = np.zeros((epochs+1))
  loss[0] = calculate_loss(train_images, weights, train_labels)

  for i in tqdm(range(epochs)):
    k =  N // B
    for j in range(k):
      #selecting random inputs/output batch
      batch = np.random.choice(range(N),B,replace=False)
      batch_input = train_images[batch]
      batch_output = train_labels[batch]
      #calculating predictions for this batch
      y_hat = calculate_output(batch_input, weights)

      #calculate gradient for weights
      gradient = (batch_input.T @ (y_hat - batch_output[:,np.newaxis]))

      #update weights
      weights -= step_size * gradient

    #calculate new loss
    loss[i+1] = calculate_loss(train_images, weights, train_labels)

    #calculate time passed for nth pass (n=time_steps)
    times.append(time.time() - start)

  return weights, loss, times

In [ ]:
def ADAM_regression(epochs, B, time_steps, train_images, train_labels, learning_rate = 0.001,beta1=0.9, beta2=0.999, epsilon=1e-8):
  """
  ADAM optimization for logistic regression.

  Parameters:
  - epochs (int): Number of training epochs.
  - B (int): Batch size for each iteration.
  - time_steps (int): Number of time steps to record for timing.
  - train_images (numpy.ndarray): Input features for training.
  - train_labels (numpy.ndarray): Output labels for training.
  - learning_rate (float, optional): Learning rate for ADAM. Default is 0.001.
  - beta1 (float, optional): Exponential decay rate for the first moment estimates. Default is 0.9.
  - beta2 (float, optional): Exponential decay rate for the second moment estimates. Default is 0.999.
  - epsilon (float, optional): Small constant to avoid division by zero. Default is 1e-8.

  Returns:
  - weights (numpy.ndarray): Learned weights after training.
  - loss (numpy.ndarray): Array of losses at each epoch.
  - times (list): List of time passed for each time step.
  """

  #calculating output by logistic regression
  def calculate_output(x, w):
    return (1 + np.exp(-x @ w))**(-1)

  # loss calculations using
  def calculate_loss(x, w, y):
    y_hat = calculate_output(x, w)

    epsilon = 1e-15
    y_hat = np.clip(y_hat, epsilon, 1 - epsilon)
    return -1/len(y_hat) * np.sum(y.T @ np.log(y_hat) + (1-y).T @ np.log(1 - y_hat))

  start = time.time()
  times = [0]
  weights = np.zeros((train_images.shape[1], 1))
  N = train_images.shape[0]
  loss = np.zeros((epochs+1))
  loss[0] = calculate_loss(train_images, weights, train_labels)

  m = 0
  v = 0

  for i in tqdm(range(epochs)):
    k =  N // B
    for j in range(k):
      #selecting random inputs/output batch
      batch = np.random.choice(range(N),B,replace=False)
      batch_input = train_images[batch]
      batch_output = train_labels[batch]
      #calculating predictions for this batch
      y_hat = calculate_output(batch_input, weights)


      #calculate gradient for weights
      gradient = (batch_input.T @ (y_hat - batch_output[:,np.newaxis]))

      #update moving averages
      m = beta1 * m + (1 - beta1) * gradient
      v = beta2 * v + (1 - beta2) * gradient**2

      # bias correction
      m_hat = m / (1 - beta1**(i * k + j + 1))
      v_hat = v / (1 - beta2**(i * k + j + 1))

      # update weights using ADAM update rule
      weights -= learning_rate * m_hat / (np.sqrt(v_hat) + epsilon)

    loss[i+1] = calculate_loss(train_images, weights, train_labels)

    times.append(time.time() - start)

  return weights, loss, times

In [ ]:
#A simple method to sanitize the data gathered by the AI-SGD experiments in R
def reorganize_df(df : pd.DataFrame, row_names : list[str], total_passes : int,step_size : int, add_zeros=False):
  df = pd.DataFrame(np.array(df).reshape((len(row_names),-1)))
  df.index = row_names
  if add_zeros:
    col_names = list(range(step_size,total_passes+1, step_size))
    df.columns = col_names
    df[0] = 0
    df = df.loc[:,list(range(0, total_passes+1, step_size))]
  else:
    col_names = list(range(0,total_passes+1, step_size))
    df.columns = col_names
  return df

# MNIST dataset

In [ ]:
from tensorflow.keras.datasets import mnist

# Load and preprocess the MNIST dataset
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images.reshape((60000, -1)).astype('float32') / 255
test_images = test_images.reshape((10000, -1)).astype('float32') / 255

# Create binary labels indicating whether the digit is 9 or not
train_labels = (train_labels == 9).astype('float32')
test_labels = (test_labels == 9).astype('float32')

11490434/11490434 [==============================] - 0s 0us/step


In [ ]:
#initializing experiment parameters
epochs = 60
time_step = 5
B = 1000
#running ADAM with the above parameters
adam = ADAM_regression(epochs, B, time_step, train_images, train_labels,
                       0.001, 0.9,0.99)

In [ ]:
#running SGD with the parameters
step_size = 0.0001
SGD = SGD_regression(epochs, B, time_step, train_images, train_labels, step_size)

In [ ]:
#plotting loss for ADAM and SGD
plt.plot(adam[2], adam[1], label="ADAM")
plt.plot(adam[2], SGD[1], label="SGD")
plt.xlabel("Time (s)")
plt.ylabel(r"Loss $(y-\hat{y})^2$")
plt.legend(fontsize="large")
plt.title("Logistic Regression on MNIST with SGD and ADAM")
plt.yscale("log")
plt.savefig('SGD ADAM loss.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
#plotting time for SGD and ADAM
plt.plot(range(0,epochs+1), adam[2], label="ADAM")
plt.plot(range(0,epochs+1), SGD[2], label="SGD")
plt.xlabel("Epoch")
plt.ylabel("Time (s)")
plt.legend(fontsize="large")
plt.title("Logistic Regression on MNIST for SGD and ADAM")
#plt.yscale("log")
plt.savefig('SGD ADAM time.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
ROW_NAMES = ["SGD","A-SGD","Implicit-SGD","AI-SGD"]
#Data for runs from 0-20 epochs
passes_20 = reorganize_df(
    pd.read_csv("passes_20.csv", header=None),
    ROW_NAMES, 20, 2)
errors_20 = reorganize_df(
    pd.read_csv("error_20.csv", header=None),
    ROW_NAMES, 20, 2)
times_20 = reorganize_df(
    pd.read_csv("times_20.csv", header=None),
    ROW_NAMES, 20, 2, True)

#Data for runs from 0-60 epochs
passes_60 = reorganize_df(
    pd.read_csv("passes_60.csv", header=None),
    ROW_NAMES, 60, 2)
times_60 = reorganize_df(
    pd.read_csv("times_60.csv", header=None),
    ROW_NAMES, 60, 2, True)


In [ ]:
#plotting error versus time for the 4  R SGD methods
for i in ROW_NAMES:
  plt.plot(times_20.loc[i], errors_20.loc[i], label=f"{i}")

plt.title("Time vs Error for AI-SGD and other methods")
plt.xlabel("Time (s)")
plt.ylabel("Loss")
plt.legend(fontsize="large")
plt.yscale("log")

plt.savefig('AI-SGD error.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
#plotting time-elapsed versus epochs for the four methods
for i in ROW_NAMES:
  plt.plot(passes_60.loc[i], times_60.loc[i], label=f"{i}")

plt.title("Epochs vs Time for AI-SGD and other methods")
plt.xlabel("Epochs")
plt.ylabel("Time (s)")
plt.legend(fontsize="large")

plt.savefig('AI-SGD time.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
#plotting the relative time elapsed for the four R methods and ADAM
#each one is normalized relative to their respective SGD method
for i in ROW_NAMES:
  plt.plot(passes_60.loc[i], np.array(times_60.loc[i])/np.array(times_60.loc["SGD"]), label=f"{i}")

plt.plot(range(0,epochs+1), np.array(adam[2])/np.array(SGD[2]), label="ADAM")

plt.title("Epochs vs Time Relative to SGD")
plt.xlabel("Epochs")
plt.ylabel("Times slower than SGD")
plt.legend(fontsize="large")

plt.savefig('time baseline SGD.png', dpi=300, bbox_inches='tight')

In [ ]:
#Plotting Loss vs epochs relative to SGD loss for the 4 R methods and ADAM
#each graph is normalized to its respective SGD implementation
for i in ROW_NAMES:
  plt.plot(passes_20.loc[i], np.array(errors_20.loc[i])/np.array(errors_20.loc["SGD"]), label=f"{i}")

plt.plot(range(0,21), np.array(adam[1][:21])/np.array(SGD[1][:21]), label="ADAM")

plt.title("Epochs vs loss relative to SGD loss")
plt.xlabel("Epochs")
plt.ylabel("Loss proportional to SGD loss")
plt.legend(fontsize="large")

plt.savefig('loss baseline SGD.png', dpi=300, bbox_inches='tight')

# CovType dataset

In [ ]:
from sklearn.datasets import fetch_covtype
from sklearn.model_selection import train_test_split

# Fetch the Covertype dataset
covtype = fetch_covtype()

# Access features and target variable
X = covtype.data
y = covtype.target

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = (X_train - np.min(X_train, axis=0)) / (np.max(X_train, axis=0) - np.min(X_train, axis=0))
#X_test = (X_test - np.min(X_test)) / (np.max(X_test) - np.min(X_test))

# Create binary labels indicating whether the digit is 2 or not
y_train = (y_train == 2).astype('float32')
y_test = (y_test == 2).astype('float32')

In [ ]:
#initializing experiment parameters
epochs = 60
time_step = 5
B = 5_000
step_size = 0.00002

In [ ]:
#Run SGD on CovType
cSGD = SGD_regression(epochs, B, 1, X_train, y_train, step_size)

In [ ]:
#Run ADAM on CovType
cADAM = ADAM_regression(epochs, B, 1, X_train, y_train, 0.05, 0.99, 0.999)

In [ ]:
#plotting loss
plt.plot(cADAM[2], cADAM[1], label="ADAM")
plt.plot(cSGD[2], cSGD[1], label="SGD")
plt.xlabel("Time (s)")
plt.ylabel(r"Loss $(y-\hat{y})^2$")
plt.legend(fontsize="large")
plt.title("Logistic Regression on CovType with SGD and ADAM")
plt.yscale("log")
plt.savefig('Cov - SGD ADAM loss.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
#plotting time
plt.plot(range(0,epochs+1), cADAM[2], label="ADAM")
plt.plot(range(0,epochs+1), cSGD[2], label="SGD")
plt.xlabel("Epoch")
plt.ylabel("Time (s)")
plt.legend(fontsize="large")
plt.title("Logistic Regression on CovType for SGD and ADAM")
#plt.yscale("log")
plt.savefig('Cov - SGD ADAM time.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
ROW_NAMES = ["SGD","A-SGD","Implicit-SGD","AI-SGD"]
#Data for runs from 0-60 epochs
c_passes = reorganize_df(
    pd.read_csv("passes.csv", header=None),
    ROW_NAMES, 60, 2)
c_errors = reorganize_df(
    pd.read_csv("error 4.csv", header=None, ),
    ROW_NAMES, 60, 2)
c_times = reorganize_df(
    pd.read_csv("times 4.csv", header=None),
    ROW_NAMES, 60, 2, True)

In [ ]:
#plotting the relative time elapsed for the four R methods and ADAM
#each one is normalized relative to their respective SGD method
for i in ROW_NAMES:
  plt.plot(c_passes.loc[i], np.array(c_times.loc[i])/np.array(c_times.loc["SGD"]), label=f"{i}")

plt.plot(range(0,epochs+1), np.array(cADAM[2])/np.array(cSGD[2]), label="ADAM")

plt.title("Epochs vs Time Relative to SGD")
plt.xlabel("Epochs")
plt.ylabel("Times slower than SGD")
plt.legend(fontsize="large")

plt.savefig('Cov - time baseline SGD.png', dpi=300, bbox_inches='tight')

In [ ]:
#Plotting Loss vs epochs relative to SGD loss for the 4 R methods and ADAM
#each graph is normalized to its respective SGD implementation
for i in ROW_NAMES:
  plt.plot(c_passes.loc[i], np.array(c_errors.loc[i])/np.array(c_errors.loc["SGD"]), label=f"{i}")

plt.plot(range(0,epochs+1), np.array(cADAM[1])/np.array(cSGD[1]), label="ADAM")

plt.title("Epochs vs loss relative to SGD loss")
plt.xlabel("Epochs")
plt.ylabel("Loss proportional to SGD loss")
plt.legend(fontsize="large")

plt.savefig('Cov - loss baseline SGD.png', dpi=300, bbox_inches='tight')